# Tutorial for Faulting

This file is meant to demonstrate the use of gstlearn for Faulting. It is recalled that Faulting is defined in 2-D.

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt

## Defining the Information

Setting some global variables

In [ ]:
ndim = 2
gl.ASpaceObject_defineDefaultSpace(gl.ESpaceType.SPACE_RN, ndim)

Defining the output grid

In [ ]:
nx = [100, 100]
grid = gl.DbGrid.create(nx)

Defining the data points

In [ ]:
nech = 30
data = gl.Db.createFromDbGrid(nech, grid)

Defining a Model which will serve in simulating the value at the data points.

In [ ]:
range = 40
model = gl.Model.createFromParam(gl.ECov.CUBIC, range)

Simulate the information at the data points

In [ ]:
err = gl.simtub(None, data, model, namconv = gl.NamingConvention("Data"))

Define the set of Faults.

In [ ]:
faults = gl.Faults()
faults.addFault(gl.PolyLine2D.create(x = [5., 10., 20.], y = [5., 25., 50.1]))
faults.addFault(gl.PolyLine2D.create(x = [50., 90.], y = [60., 70.]))
faults.addFault(gl.PolyLine2D.create(x = [20., 80., 80., 20.], y = [10., 40., 10., 10.]))

Define the Moving Neighborhood (with a maximum of 'nmaxi' samples per neighborhood) an attach the Faults.

In [ ]:
nmaxi = 10;
neighM = gl.NeighMoving.create(nmaxi = nmaxi, faults = faults)

Plot the information

In [ ]:
ax = gp.point(data, figsize=[8,8],title="Information")
ax = gp.fault(faults, ax=ax)

## Calculating the Variogram

We define the parameters for the calculation of an omni-directional variogram.
In a first trial, we compute the variogram without taking faults into account.

In [ ]:
varioparam = gl.VarioParam.createOmniDirection(npas=10, dpas=5)
vario = gl.Vario.create(varioparam, data)
err = vario.compute()

Represent the variogram with the number of pairs attached to each lag.

In [ ]:
ax = gp.vario(vario, show_pairs=True)

Performing the same variogram calculation, taking the fault into account

In [ ]:
varioparam = gl.VarioParam.createOmniDirection(npas=10, dpas=5)
varioparam.addFaults(faults)
vario = gl.Vario.create(varioparam, data)
err = vario.compute()

Represent the variogram again with the number of pairs attached to each lag.

In [ ]:
ax = gp.vario(vario, show_pairs=True)

## Performing the Estimation

In [ ]:
err = gl.kriging(data, grid, model, neighM)

Plotting the result

In [ ]:
ax = gp.grid(grid, name="*estim", figsize=[8,8],
             title="Kriging Estimate with Faults")
ax = gp.point(data, ax=ax)
ax = gp.fault(faults, ax=ax)

In [ ]:
ax = gp.grid(grid, name="*stdev", figsize=[8,8],
             title="St. Dev. with Faults")
ax = gp.point(data, ax=ax)
ax = gp.fault(faults, ax=ax)